# Ingesta y chunking

**Importe de Liberias**

In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName("RAG_Ingesta").getOrCreate()

Carga de Documento

Este documento fue extraido de la documentación oficial https://learn.microsoft.com/es-es/azure/azure-sql/database/sql-database-paas-overview?view=azuresql. 

Nota: Solo extraemos 12 paginas, esto por motivos de recursos.

In [0]:
%pip install pdfplumber
import pdfplumber

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


## Carga de Documento

El documento se encuentra almacenado en un volumen en capa bronce

In [0]:
document_path = "/Volumes/corona/bronce/doc/Azure SQL Database _ Microsoft Learn.pdf"

In [0]:
text = ""
with pdfplumber.open(document_path) as pdf:
    for page in pdf.pages:
        text += page.extract_text() + "\n\n"

print(text[:1000])

24/11/25, 9:34 ¿Qué es el servicio Azure SQL Database? - Azure SQL Database | Microsoft Learn
¿Qué es Azure SQL Database?
Se aplica a: Azure SQL Database
En este artículo se proporciona una introducción a Azure SQL Database, un motor de base de datos de plataforma como servicio
(PaaS) totalmente administrado que se encarga de la mayoría de las funciones de administración de bases de datos, como las de
actualizar, aplicar revisiones, crear copias de seguridad y supervisar sin intervención del usuario.
７ Nota:
Pruebe Azure SQL Database de forma gratuita y obtenga 100 000 segundos de núcleo virtual de proceso sin servidor y 32 GB
de almacenamiento cada mes.
Información general
Azure SQL Database se ejecuta siempre en la última versión estable del motor de base de datos de SQL Server y en un sistema
operativo revisado con el 99,99 % de disponibilidad . Las funcionalidades PaaS integradas en Azure SQL Database permiten
centrarse en las actividades de administración y optimización de bases d

## Chunks por longitud

In [0]:
#voy a utilizar un tamaño pequeño para evitar posibles errores de recursos
chunk_size = 600
chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]


In [0]:
#creo el dataframe de chunks
df_chunks = spark.createDataFrame(
    [(i, chunk) for i, chunk in enumerate(chunks)],
    ["chunk_id", "chunk_text"]
)

## Almaceno de dataframe en silver

In [0]:
df_chunks.write.mode("overwrite").parquet("dbfs:/Volumes/corona/silver/chunk")
df_chunks.show(truncate=100)

+--------+-----------------------------------------------------------------------------------------------------+
|chunk_id|                                                                                           chunk_text|
+--------+-----------------------------------------------------------------------------------------------------+
|       0| 24/11/25, 9:34 ¿Qué es el servicio Azure SQL Database? - Azure SQL Database | Microsoft Learn\n¿Q...|
|       1| al de proceso sin servidor y 32 GB\nde almacenamiento cada mes.\nInformación general\nAzure SQL D...|
|       2| as soluciones de Azure. SQL Database puede ser la opción adecuada para una variedad de aplicacion...|
|       3| ublican primero en Azure SQL Database y,\ndespués, en el propio SQL Server. Las funcionalidades d...|
|       4| elo de compra basado en DTU. SQL Database es un servicio totalmente administrado que\nofrece alta...|
|       5| ompra\nSQL Database ofrece los siguientes modelos de compra:\nEl modelo de compra bas